# Dataframe Processing

In [1]:
# Get the data from the sample_data folder

import os

In [2]:
# Get the data from the sample_data folder

def get_data_path(filename):
    """
    Get the path to the data file in the sample_data folder.
    :param filename: The name of the file to get.
    :return: The path to the file.
    """
    return os.path.join(os.path.dirname(__file__), 'sample_data', filename)


In [5]:
# Get the sample_data/4_5_25 SP 25 Manan Test Sheet - Lecture Quiz 4_ Iteration.csv file
def get_test_sheet():
    """
    Get the path to the test sheet.
    :return: The path to the test sheet.
    """
    return get_data_path('4_5_25 SP 25 Manan Test Sheet - Lecture Quiz 4_ Iteration.csv')

In [6]:
get_test_sheet()

NameError: name '__file__' is not defined